In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sample data
data = pd.DataFrame({
    'category': ['Electronics', 'Clothing'] * 50,
    'region': ['North', 'South', 'East', 'West'] * 25,
    'sales_volume': [100, 200, 150, 300] * 25,
    'revenue': [5000, 8000, 6000, 12000] * 25,
    'customer_rating': [4.5, 3.8, 4.2, 4.0] * 25,
    'month': ['Jan', 'Feb', 'Mar', 'Apr'] * 25
})

# Map region names to valid CSS colors
region_colors = {
    'North': 'blue',
    'South': 'green',
    'East': 'orange',
    'West': 'red'
}
data['region_color'] = data['region'].map(region_colors)

# Create subplots: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Sales vs Revenue", "Seasonal Trends Heatmap"),
    column_widths=[0.5, 0.5]
)

# 📈 Scatter Plot (left)
scatter = go.Scatter(
    x=data['sales_volume'],
    y=data['revenue'],
    mode='markers',
    marker=dict(color=data['region_color'], size=10),
    text=[f"Region: {r}<br>Rating: {rt}" for r, rt in zip(data['region'], data['customer_rating'])],
    hoverinfo='text+x+y',
    name='Sales Data'
)
fig.add_trace(scatter, row=1, col=1)

# 🔥 Heatmap (right)
heatmap_data = data.groupby(['category', 'month'])['sales_volume'].mean().unstack().fillna(0)
heatmap = go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='YlGnBu',
    name='Seasonal Trends'
)
fig.add_trace(heatmap, row=1, col=2)

# 📅 Dropdown Menu for Month Filter (affects scatter plot only)
months = sorted(data['month'].unique())
dropdown_buttons = []

for month in months:
    filtered = data[data['month'] == month]
    dropdown_buttons.append(dict(
        label=month,
        method='update',
        args=[{
            'x': [filtered['sales_volume'], heatmap_data.columns],
            'y': [filtered['revenue'], heatmap_data.index],
            'text': [[f"Region: {r}<br>Rating: {rt}" for r, rt in zip(filtered['region'], filtered['customer_rating'])], None],
            'marker': [{'color': filtered['region'].map(region_colors), 'size': 10}, None]
        }]
    ))

# Expand axes for better visibility
fig.update_xaxes(range=[0, 350], row=1, col=1)
fig.update_yaxes(range=[0, 13000], row=1, col=1)

# Final layout
fig.update_layout(
    title="📊 E-commerce Sales Dashboard",
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        y=1.15,
        xanchor='left',
        yanchor='top'
    )],
    height=600,
    width=1000
)

fig.show()